<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_glove_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Libraries, setting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [12]:
import warnings
import pickle

import numpy as np
import pandas as pd
import json

import pprint

from tensorflow.keras import preprocessing

from numpy import array
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# we will store the params as we go along in this object
params = {}
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"
model_path = "/content/drive/My Drive/AIML-MRC-Capstone/models/"
tensorboard_logpath  = "/content/drive/My Drive/AIML-MRC-Capstone/models/tensorboard-logs/"
token_file = "tokenizer.pkl"

### 2.3 Load Tokenizer

In [5]:
tfile = model_path + token_file
print(tfile)

/content/drive/My Drive/AIML-MRC-Capstone/models/tokenizer.pkl


In [0]:
with open(tfile,"rb") as infile:
    tknizer = pickle.load(infile)



In [7]:
tknizer.word_index['how']

39

In [0]:
squad_df = pd.read_csv(project_path+'squad_data_final_withstopwords_withpunctuations.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.head(5)

In [13]:
from tqdm import tqdm

tokenizer_num_words = 80000
tokenizer = preprocessing.text.Tokenizer(num_words=tokenizer_num_words)

for text in tqdm([squad_df['clean_context'], squad_df['clean_question']]):  
  tokenizer.fit_on_texts(text.values)

# # total tokenizer words
print(len(tokenizer.word_index)) 

100%|██████████| 2/2 [00:12<00:00,  6.21s/it]

82505


In [14]:
tokenizer.word_index['how']

79

In [0]:
# total tokenizer words
vocab_size = len(tknizer.word_index)

In [31]:
vocab_size

100850

### Download Glove Model

In [0]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-06-10 08:05:41--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-06-10 08:05:41--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-06-10 08:05:41--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [0]:
#Check if embeddings have been downloaded
!ls -l

total 1833804
drwx------ 4 root root       4096 Jun 10 08:03 drive
-rw-r--r-- 1 root root 1877800501 Oct 25  2015 glove.42B.300d.zip
drwxr-xr-x 1 root root       4096 May 29 18:19 sample_data


In [0]:
#unzip the file, we get multiple embedding files. We can use either one of them
!unzip glove.42B.300d.zip

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
!ls -l

total 6741064
drwx------ 4 root root       4096 Jun 10 08:03 drive
-rw-rw-r-- 1 root root 5025028820 Oct 24  2015 glove.42B.300d.txt
-rw-r--r-- 1 root root 1877800501 Oct 25  2015 glove.42B.300d.zip
drwxr-xr-x 1 root root       4096 May 29 18:19 sample_data


### Saving Embedding Matrix

In [0]:
#Install Gensim
!pip install gensim --quiet

In [0]:
import csv
glovewords = pd.read_csv("/content/glove.42B.300d.txt", sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [0]:
glovewords.loc['using'].values

array([-4.8024e-01, -3.1791e-01, -1.3066e-01, -3.7725e-01,  1.4999e-01,
       -8.3056e-02, -3.9617e+00,  1.0422e+00,  1.6753e-01, -4.4412e-01,
        2.5729e-01, -6.9419e-02, -1.6633e-01, -1.6982e-01,  2.5493e-01,
       -2.2274e-01, -5.2021e-01,  2.5322e-02,  3.6309e-02, -1.0757e-01,
        2.9680e-02, -2.1614e-01,  2.0985e-01, -1.3535e-01,  4.6872e-01,
       -6.1247e-01,  1.0688e-01, -5.1693e-01, -6.1304e-01, -4.8695e-01,
        6.4884e-02, -3.7728e-01, -2.7324e-01, -1.5761e-01,  2.3469e-01,
       -3.4595e-01, -7.5058e-02,  2.0815e-02,  1.1814e-01,  8.6750e-02,
       -5.2511e-02, -3.5294e-01, -2.4328e-02, -1.7673e-01,  1.7192e-01,
        2.4108e-01,  2.1050e-01, -2.4947e-01,  4.1514e-01,  5.9159e-02,
        3.4335e-01, -6.9559e-01, -1.5854e-02, -5.2379e-01,  1.7853e-01,
        4.5934e-01, -3.1204e-01,  4.3102e-01, -1.0625e-01, -4.7017e-02,
       -9.7047e-02,  2.7452e-01, -4.8079e-01,  8.0425e-02, -5.2430e-02,
       -1.0408e-01, -3.6195e-01,  1.2116e-02,  7.8531e-02, -4.13

In [0]:
# +1 for zero embedding
#glovewords.loc['using'].values

GLOVE = 300
word_array = glovewords.index.values

embedding_matrix = np.zeros(vocab_size + 1,GLOVE)

print("Embedding matrix shape is ",embedding_matrix.shape)
len(embedding_matrix)

for word, i in tknizer.word_index.items():
    if i > (vocab_size + 1):
        break
    if word in word_array:
        embedding_matrix[i] = glovewords.loc[word].values
        
print("Saving the embedding matrix for future use...")

with open(model_path + "glove300dembedmatrix.pkl","wb") as f:
    pickle.dump(embedding_matrix,f)


    

Embedding matrix shape is  (100851, 300)


### Universal Sentence encoder 

In [0]:
import tensorflow_hub as hub

In [17]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [18]:
sz = len(tokenizer.word_index) + 1
embed_matrix_uni = np.zeros((sz,512))

print("Embedding matrix shape is ",embed_matrix_uni.shape)
len(embed_matrix_uni)

Embedding matrix shape is  (82506, 512)


82506

In [0]:
#tknizer.word_index.items()
#embed_mat1

for word, i in tokenizer.word_index.items():
  if i > (sz):
        break
  embed_matrix_uni[i] = embed([word])



In [0]:
with open(model_path + "universalembedmatrix.pkl","wb") as f:
    pickle.dump(embed_matrix_uni,f)

# **<font color="GREEN">END OF THE NOTEBOOK </font>**